In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from collections import ChainMap

import json

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
%config InlineBackend.figure_format='retina'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'GeForce RTX 2070 SUPER'

In [2]:
og_df = pd.read_csv("gvdb-aggregated-db\Articles-with-extracted-info.tsv", sep='\t')
og_df.head()

,Article_url,Article_title,Full_text,Json,Publication_date
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00


In [3]:
print(og_df.iloc[0] == og_df.iloc[0])

Article_url         True
Article_title       True
Full_text           True
Json                True
Publication_date    True
Name: 0, dtype: bool


In [4]:
#df = og_df.drop_duplicates()

In [5]:
df = og_df
df

,Article_url,Article_title,Full_text,Json,Publication_date
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway...","{""circumstances"":{""number-of-shots-fired"":{""en...",2014-07-29 00:00:00+00:00
23900,gold-data16.com,23 year old man shot dead in nightclub,"OAKLAND, CA -- A 23-year-old Hispanic man was ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2014-10-21 00:00:00+00:00
23901,http://gold-data17.com/,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00


In [6]:
df.shape

(23903, 5)

In [7]:
df = df.join(df['Json'].apply(json.loads).apply(pd.Series))

In [8]:
df.columns = df.columns.str.strip().str.lower().str.replace('-', '_')

In [9]:
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v..."
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v..."
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v..."
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v..."
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v..."
...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIndex': 182, 'startIndex': 179, ..."
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway...","{""circumstances"":{""number-of-shots-fired"":{""en...",2014-07-29 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 1073, '...","{'city': {'endIndex': 59, 'startIndex': 49, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIn

In [10]:
df_s = pd.DataFrame(df.shooter_section)

In [11]:

df.shooter_section[2][0].get('race').get('value')
# sns.countplot(x='race', data = df.victim_section[0][0].get('race').get('value'))

'white'

In [12]:
from collections import ChainMap
multiDct_to_single=[]
for i in range(len(df['shooter_section'])):
    qq=dict(ChainMap(*df['shooter_section'][i][::-1]))
    multiDct_to_single.append(qq)
df_flat= pd.DataFrame(multiDct_to_single)
df_flat

,age,gender,name,race
0,"{'endIndex': 22, 'startIndex': 20, 'value': '26'}",Male,"{'endIndex': 45, 'startIndex': 63, 'value': 'J...","{'endIndex': -1, 'startIndex': -1, 'value': ''}"
1,"{'endIndex': 22, 'startIndex': 20, 'value': '26'}",Male,"{'endIndex': 45, 'startIndex': 63, 'value': 'J...","{'endIndex': -1, 'startIndex': -1, 'value': ''}"
2,"{'endIndex': 65, 'startIndex': 54, 'value': '2...",Male,"{'endIndex': -1, 'startIndex': -1, 'value': ''}","{'endIndex': 71, 'startIndex': 66, 'value': 'w..."
3,"{'endIndex': 56, 'startIndex': 54, 'value': '23'}",Male,"{'endIndex': -1, 'startIndex': -1, 'value': ''}","{'endIndex': 71, 'startIndex': 66, 'value': 'w..."
4,"{'endIndex': 22, 'startIndex': 20, 'value': '26'}",Male,"{'endIndex': 45, 'startIndex': 32, 'value': 'J...","{'endIndex': -1, 'startIndex': -1, 'value': ''}"
...,...,...,...,...
23898,NaN,NaN,NaN,NaN
23899,NaN,NaN,NaN,NaN
23900,NaN,NaN,NaN,NaN
23901,"{'endIndex': 100, 'startIndex': 98, 'value': '...",Male,"{'endIndex': -1, 'startIndex': -1, 'value': ''}","{'endIndex': 115, 'startIndex': 110, 'value': ..."


In [13]:
def func(dict_):
    h=defaultdict(list)
    for j in dict_:
        if type(j)!=str:
            for i in j.items():
                h[i[0]].append(i[1])
    return h


t_=[]
g=defaultdict(list)
for i in range(len(df_flat['race'])):
    tdict = df_flat['race'][i]
    if not isinstance(tdict,float):
        t_.append(tdict.get('value').strip().lower())
    else:
        t_.append("NaN")
df_race = pd.DataFrame(t_)
df_race.columns = ["race"]
df_race

,race
0,
1,
2,white
3,white
4,
...,...
23898,NaN
23899,NaN
23900,NaN
23901,white


In [14]:
# sLength = len(df['article_title'])
#df = df.assign(df_race=pd.Series(np.random.randn(sLength)).values)
#df = df.join(df_race.apply(pd.Series))
df['race'] = df_race.values
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section,race
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v...",white
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v...",white
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIndex': 182, 'startIndex': 179, ...",NaN
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway...","{""circumstances"":{""number-of-shots-fired"":{""en...",2014-07-29 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 1073, '...","{'city': {'endIndex': 59, 'startIndex': 49, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/vict

In [15]:
#    if type(df_flat['race'][i])!=str:
#        t_.append(func(df_flat['race'][i]))
#    
#    else:
#        tups=tuple(zip(df_flat['race'][0][0].keys() ,
#                               ['Nope']*len(df_flat['race'][0][0].keys())))
#        for n in tups:
#            g[n[0]].append(n[1])
#        t_.append(g)

#print(pd.DataFrame(t_).url[0])
#print('--------------------')
#print(pd.DataFrame(t_).url[1])

#url_expand_=pd.DataFrame(t_)
#url_expand_

In [16]:
# shooters_list = list(df['shooter_section'].items())

In [17]:
# print(type(shooters_list))
# thislist = shooters_list[99][1]
# print(thislist)
# thislist[0]['race'].get('value')

In [18]:
# races = []
# for case in shooters_list:
    ## info_list = shooters_list[case][1]
    ## races.append(info_list[0]['race'].get('value'))
    ## print(case[1])
    # temp = case[1]
    # print(temp[0]['race'].get('value'))

In [19]:
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section,race
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v...",white
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v...",white
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIndex': 182, 'startIndex': 179, ...",NaN
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway...","{""circumstances"":{""number-of-shots-fired"":{""en...",2014-07-29 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 1073, '...","{'city': {'endIndex': 59, 'startIndex': 49, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/vict

In [20]:
import nltk

In [21]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\khanda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\khanda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\khanda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\khanda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\khanda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_dat

True

In [22]:
df['tokenized_title'] = df["article_title"].apply(word_tokenize)

In [23]:
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section,race,tokenized_title
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,..."
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,..."
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIndex': 182, 'startIndex': 179, ...",NaN,"[Woman, Dies, After, Being, Shot, in, Head, Al..."
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway..."

In [24]:
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section,race,tokenized_title
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,..."
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,..."
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",,"[Man, shoots, self, accidentally, while, in, car]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,[],"[{'age': {'endIndex': 182, 'startIndex': 179, ...",NaN,"[Woman, Dies, After, Being, Shot, in, Head, Al..."
23899,gold-data15.com,Bridgeport man murdered in 9th homocide this year,"Bridgeport, Connecticut -- Christopher Pettway..."

In [25]:
v = TfidfVectorizer()
x = v.fit_transform(df['article_title'])

In [26]:
df['title_vect'] = list(x)

In [27]:
df

,article_url,article_title,full_text,json,publication_date,circumstances,date_and_time,radio1,radio2,radio3,shooter_section,victim_section,race,tokenized_title,title_vect
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]","(0, 1721)\t0.4563040367696755\n (0, 4737)\t..."
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 451, 's...","{'city': {'endIndex': 8, 'startIndex': 0, 'val...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'No', 'Drugs (other ...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': 22, 'startIndex': 32, 'v...",,"[Man, shoots, self, accidentally, while, in, car]","(0, 1721)\t0.4563040367696755\n (0, 4737)\t..."
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Yes', 'Drugs (other...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 65, 'startIndex': 54, 'v...","[{'age': {'endIndex': 65, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,...","(0, 4232)\t0.3975455962537428\n (0, 2105)\t..."
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': 122, 's...","{'city': {'endIndex': 11, 'startIndex': 0, 'va...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 56, 'startIndex': 54, 'v...","[{'age': {'endIndex': 56, 'startIndex': 54, 'v...",white,"[Man, accidentally, shoots, himself, cleaning,...","(0, 4232)\t0.3975455962537428\n (0, 2105)\t..."
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 13, 'startIndex': 12, 'v...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned by the victim/victims ...,"[{'age': {'endIndex': 22, 'startIndex': 20, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",,"[Man, shoots, self, accidentally, while, in, car]","(0, 1721)\t0.4563040367696755\n (0, 4737)\t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23898,http://www.nbcphiladelphia.com/news/local/Woma...,Woman Dies After Being Shot in Head Along Nice...,A woman has been killed after being shot in th...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00,"{'number-of-shots-fired': {'endIndex': -1, 'st...","{'city': {'endIndex': 153, 'startIndex': 140, ...",{'The firearm was used during another crime.':...,"{'Alcohol was involved.': 'Not mentioned', 'Dr...",{'The firearm was owned 

In [28]:
print(df['title_vect'][0])

  (0, 1721)	0.4563040367696755
  (0, 4737)	0.13795119470958206
  (0, 9904)	0.452601971846722
  (0, 306)	0.4100683058394725
  (0, 8294)	0.44386713360405866
  (0, 8435)	0.3976772299113978
  (0, 5922)	0.21115473484725697


In [29]:
(df.race == 'african-american').sum()

10

In [30]:
((df.race != 'white') & (df.race !='') & (df.race !='NaN')).sum()

925

In [31]:
((df.race == 'hispanic') | (df.race == 'asian') | (df.race == 'mexican') | (df.race == 'black') | (df.race == 'African American') | (df.race == 'Asian-American') | (df.race == 'Afrian-American') | (df.race == 'Asian-American') ).sum()

491

In [32]:
df['race'] = df.race.str.lower()
aa_arr = []
asian_arr = []
hisp_arr = [] 
white_arr = []
empty_arr = []

In [33]:
aa_arr = (df.index[df.race == 'black']) | (df.index[df.race == 'african']) | (df.index[df.race == 'african-american']) | (df.index[df.race == 'african american'])
asian_arr = (df.index[df.race == 'asian']) | (df.index[df.race == 'asian-american']) | (df.index[df.race == 'asian american']) | (df.index[df.race == 'chinese']) | (df.index[df.race == 'chinese-american']) | (df.index[df.race == 'chinese american']) | (df.index[df.race == 'vietnamese']) | (df.index[df.race == 'korean'])  
hisp_arr = (df.index[df.race == 'hispanic']) | (df.index[df.race == 'mexican'])
white_arr = (df.index[df.race == 'white']) | (df.index[df.race == 'caucasian'])
empty_arr = (df.index[df.race == 'nan']) | (df.index[df.race == ''])

<ipython-input-33-6661ce9a9c2f>:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  aa_arr = (df.index[df.race == 'black']) | (df.index[df.race == 'african']) | (df.index[df.race == 'african-american']) | (df.index[df.race == 'african american'])
<ipython-input-33-6661ce9a9c2f>:2: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  asian_arr = (df.index[df.race == 'asian']) | (df.index[df.race == 'asian-american']) | (df.index[df.race == 'asian american']) | (df.index[df.race == 'chinese']) | (df.index[df.race == 'chinese-american']) | (df.index[df.race == 'chinese american']) | (df.index[df.race == 'vietnamese']) | (df.index[df.race == 'korean'])
<ipython-input-33-6661ce9a9c2f>:3: FutureWarning: Index.__or__ operating as a set operation

In [34]:
# for row in df['race']:
    # if ('black' in row.lower()):
        